In [27]:
# from tpk4170.visualization import Ur5Visualizer
from dataclasses import dataclass
import modern_robotics as mr
import numpy as np
import sympy as sp

np.set_printoptions(suppress=True)

PI_HALF = np.pi / 2
PI = np.pi


In [28]:
@dataclass
class DHLink:
    a: float
    alpha: float
    d: float
    theta: float

    def matrix(self) -> np.array:
        ct = np.cos(self.theta)
        st = np.sin(self.theta)
        ca = np.cos(self.alpha)
        sa = np.sin(self.alpha)
        return np.array(
            [
                [ct, -st * ca, st * sa, self.a * ct],
                [st, ct * ca, -ct * sa, self.a * st],
                [0.0, sa, ca, self.d],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )


In [29]:
class KR6:
    d1, d2, d3, d4 = 400, 455, 420, 80
    a1, a2 = 25, 35
    theta_zero = np.zeros((6, 1))
    
    def __init__(self):
        self._M = np.array(
            [
                [0,  0,  1, self.a1+self.d2+self.d3+self.d4],
                [0, -1,  0,                               0],
                [1,  0,  0,                 self.a2+self.d1],
                [0,  0,  0,                               1]
            ]
        )

        self._Slist = np.array(
            [
                [0, 0, 1,                  0,               0,                       0],
                [0, 1, 0,           -self.d1,               0,                 self.a1],
                [0, 1, 0,           -self.d1,               0,         self.a1+self.d2],
                [1, 0, 0,                  0, self.a2+self.d1,                       0],
                [0, 1, 0, -(self.d1+self.a2),               0, self.a1+self.d2+self.d3],
                [1, 0, 0,                  0, self.a2+self.d1,                       0]
            ]
        ).T

        T_BS = np.linalg.inv(mr.FKinSpace(self._M, self._Slist, self.theta_zero))
        self._Blist = mr.Adjoint(T_BS) @ self._Slist

    def fkin(self, theta):
        return mr.FKinSpace(self._M, self._Slist, theta)


In [30]:
kr6 = KR6()

In [31]:
theta = np.array([[0],
                  [0],
                  [PI_HALF],
                  [0],
                  [0],
                  [0]])

alpha = np.array([[0],
                  [-PI_HALF],
                  [0],
                  [-PI_HALF],
                  [PI_HALF],
                  [-PI_HALF]])

d = np.array([[0],
              [400],
              [0],
              [0],
              [420],
              [80]])

a = np.array([[0],
              [25],
              [455],
              [35],
              [0],
              [0]])

dhList = np.concatenate((a, alpha, d, theta), axis=1)
dhList


array([[  0.        ,   0.        ,   0.        ,   0.        ],
       [ 25.        ,  -1.57079633, 400.        ,   0.        ],
       [455.        ,   0.        ,   0.        ,   1.57079633],
       [ 35.        ,  -1.57079633,   0.        ,   0.        ],
       [  0.        ,   1.57079633, 420.        ,   0.        ],
       [  0.        ,  -1.57079633,  80.        ,   0.        ]])

In [32]:
l1 = DHLink(*dhList[0])
l2 = DHLink(*dhList[1])
l3 = DHLink(*dhList[2])
l4 = DHLink(*dhList[3])
l5 = DHLink(*dhList[4])
l6 = DHLink(*dhList[5])

In [33]:
fk_dh_zero = (
    l1.matrix()
    @ l2.matrix()
    @ l3.matrix()
    @ l4.matrix()
    @ l5.matrix()
    @ l6.matrix()
)
fk_dh_zero


array([[   0.,   -0.,   -1., -395.],
       [   0.,   -1.,    0.,   80.],
       [  -1.,   -0.,   -0.,  -90.],
       [   0.,    0.,    0.,    1.]])

In [34]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from pythreejs import Object3D
from transformations import quaternion_from_matrix

ModuleNotFoundError: No module named 'tpk4170'

In [ ]:
class DHFrame(Axes):
    def __init__(self, trf):
        Axes.__init__(self, 0.1)
        self.quaternion = np.roll(quaternion_from_matrix(trf), -1).tolist()
        self.position = (trf[:3, 3]).tolist()

In [ ]:
viewer = Viewer(background="white")


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [ ]:
viewer.add(Grid())

In [ ]:
T01 = l1.matrix()
T02 = l1.matrix() @ l2.matrix()
T03 = l1.matrix() @ l2.matrix() @ l3.matrix()
T04 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix()
T05 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix() @ l5.matrix()
T06 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix() @ l5.matrix() @ l6.matrix()

In [ ]:
viewer.add(Axes(0.1))

In [ ]:
viewer.add(DHFrame(T01))

In [ ]:
viewer.add(DHFrame(T02))

In [ ]:
viewer.add(DHFrame(T03))

In [ ]:
viewer.add(DHFrame(T04))

In [ ]:
viewer.add(DHFrame(T05))

In [ ]:
viewer.add(DHFrame(T06))